# 下载模型

In [ ]:
!gdown https://drive.google.com/uc?id=1-5v7gCPWaKKBtJVu0KZwvlDyEia40crr

# 克隆仓库

In [ ]:
!git clone https://github.com/adailtonaraujo/RE-BERT
!mv RE-BERT/* .

# 安装依赖

In [ ]:
!pip install -r requirements.txt

# 加载模型

In [ ]:

from re_bert_demo import re_bert_model
from re_bert_demo import extract
import numpy as np

options = ['--classifier_model_file','demo_api_8apps_v1.model']
model = re_bert_model(options)  # 加载模型

# 用例1

> I use the app to apply filters to my photos and it's really fun.

In [ ]:
review = "I use the app to apply filters to my photos and it's really fun."
predictions,requirements_extracted = extract(model,review)


print('\n\nReview: ',review)
print('Software Requirements Extracted (in each sentence):',requirements_extracted)

for item in predictions:
  sentence = item[0]
  software_requirements = item[1]
  iob_classification = item[2]
  print('\n\nSentence:',sentence)
  print('Software Requirements Tokens:',software_requirements)
  for tokens_res in iob_classification:
    token = tokens_res[0]
    confidence = np.max(tokens_res[1]['confidences'])
    iob_class = np.max(tokens_res[1]['iob'])
    iob_tag = 'O'
    if iob_class == 0: iob_tag='B'
    if iob_class == 1: iob_tag='I'
    print('Token [',token,'] is tag [',iob_tag,'] with',confidence,'confidence.')

# 用例2

> I am trying to set up a business account and change my email address and I am having the most difficulty using your app its totally ridiculous! And another thing I am unable to see prices until driver has been found.

In [ ]:
review = "I am trying to set up a business account and change my email address and I am having the most difficulty using your app its totally ridiculous! And another thing I am unable to see prices until driver has been found."
predictions,requirements_extracted = extract(model,review)


print('\n\nReview: ',review)
print('Software Requirements Extracted (in each sentence):',requirements_extracted)

for item in predictions:
  sentence = item[0]
  software_requirements = item[1]
  iob_classification = item[2]
  print('\n\nSentence:',sentence)
  print('Software Requirements Tokens:',software_requirements)
  for tokens_res in iob_classification:
    token = tokens_res[0]
    confidence = np.max(tokens_res[1]['confidences'])
    iob_class = np.max(tokens_res[1]['iob'])
    iob_tag = 'O'
    if iob_class == 0: iob_tag='B'
    if iob_class == 1: iob_tag='I'
    print('Token [',token,'] is tag [',iob_tag,'] with',confidence,'confidence.')

## 模型评估

**生成标注文件**

In [ ]:
!python extract.py --test_file test_data_Twitter.txt --classifier_model_file demo_api_8apps_v1.model --output_file Twitter_extracted_reqs.txt

**与人工标注比对**

In [ ]:
reviews_test = open('datasets_iob/test_data_Twitter.txt', 'r').readlines()
reqs_exracted =  open('Twitter_extracted_reqs.txt', 'r').readlines()
reqs_labeled =  open('datasets_iob/test_label_Twitter.txt', 'r').readlines()

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

df = pd.DataFrame()
df['review'] = reviews_test
df['labeled (humans)'] = reqs_labeled
df['extracted (re-bert)'] = reqs_exracted
df = df.replace('\n',' ', regex=True)
df